# The Genesis process

Creating a new game requires the creation of a system and planets, factions and people.

This notebook tests that process as well as documents it. This notebook tests several attributes of the generation process to ensure that the code is sound. 

In [1]:
import numpy as np
import pandas as pd

import sys, os, yaml, ssl, asyncio
import altair as alt

# mapping to the modules that make the app
sys.path.insert(0, "../..")
sys.path.insert(0, "../../app")

%load_ext lab_black

In [2]:
from app import creators
from app import objects

In [3]:
from app.creators import homeworld
from app.creators import universe

An example `data` set for testing. This is the user form that the user submits when creating a new game. 

You can get this from the template at `app\templates\app\creation\genesis.js`


In [4]:
data = {
    "label": "form",
    "name": "worldgenform",
    "objid": "0000000000001",
    "owner": "user.username",
    "username": "user.username",
    "accountid": "0000000000001",
    "conformity": 0.5,
    "constitution": 0.5,
    "literacy": 0.5,
    "aggression": 0.5,
    "num_planets": 4,
    "num_moons": 10,
    "starting_pop": 7,
    "organics": 0.5,
    "minerals": 0.5,
}

You'll also need to configuration files.

In [5]:
conf = creators.universe.configurations.get_configurations()

Each object as an `__init__` function that creates it, however not all data is spawed on init. 

## The Solar System and Planets

In [6]:
system = universe.celestials.System(data)
system

<system: ordered; 9478383991893; Hofula>

In [7]:
system.get_data()

{'objid': '9478383991893',
 'name': 'Hofula',
 'label': 'system',
 'class': 'ordered',
 'isHomeSystem': True,
 'glat': 6.205,
 'glon': 1.689,
 'gelat': 7.553}

Each object as an `__init__` function that creates it, and populates it with data available at that time. Celestial objects have dependancies on other objects. 

In [8]:
star = objects.celestials.Star(conf["star_config"], system)
star.get_data()

{'name': 'Leportfield',
 'class': 'G',
 'objid': '4945593305484',
 'label': 'star',
 'radius': 106}

Each object also inherits a `get_fundamentals()` method that ensures that needed default values are present. `get_data()` extends that functionality. This ensures that objects can alwasy interact with the graph. 

In [9]:
star.get_fundimentals()

{'name': 'Leportfield',
 'class': 'G',
 'objid': '4945593305484',
 'label': 'star'}

Procedurally generated planets chose from a list of potential types in the `conf`. To force a particular kind of outcome, reduce the options in the configuration. 

In [10]:
terrestrial_config = {"terrestrial": conf["planet_config"]["terrestrial"]}
home_planet = objects.celestials.Planet(conf=terrestrial_config, orbiting=star)
home_planet.get_data()

{'name': 'Jobangji',
 'class': 'terrestrial',
 'objid': '4208695288749',
 'label': 'planet',
 'radius': 0.784,
 'mass': 0,
 'orbitsDistance': 1.071,
 'orbitsId': '4945593305484',
 'orbitsName': 'Leportfield',
 'isSupportsLife': False,
 'isPopulated': False}

Creating a group of planets using list comprehension. Note that celestial objects have a custom `__repr__` function that makes them easy to manage.

In [11]:
planets = [
    objects.celestials.Planet(conf=conf["planet_config"], orbiting=star)
    for p in range(int(data["num_planets"]) - 1)
]
planets

[<planet: gas; 5750191789943; Salpia>,
 <planet: gas; 9708494056465; Rahrei>,
 <planet: gas; 0571213987333; Hills>]

In [12]:
moons = [
    objects.celestials.Moon(conf["moon_config"], planets)
    for p in range(int(data["num_moons"]))
]
moons

[<moon: ice; 9180000257266; Guraylarhun>,
 <moon: ice; 2071024525531; Koy>,
 <moon: ice; 9247710915138; Wi>,
 <moon: rocky; 7703474315327; Jo>,
 <moon: rocky; 4388324383549; Bangteinwan>,
 <moon: rocky; 8753484490366; Maiennal>,
 <moon: ice; 0697717419256; Sha>,
 <moon: rocky; 7505618271831; Ky>,
 <moon: rocky; 4178291394892; Kanauhal>,
 <moon: rocky; 7253055001500; Trogiopa>]

Getting the nodes and edges of each by calling the `self.orbiting` propperty.

In [13]:
moons[0].orbiting

<planet: gas; 0571213987333; Hills>

In [14]:
home_planet.orbiting

<star: G; 4945593305484; Leportfield>

Additionally, you can quickly navigate the system by referencing other objects. 

In [15]:
home_planet.orbiting.system

<system: ordered; 9478383991893; Hofula>

Getting the nodes for the graph. Sandwich all of the items together and get the data using the same generic function.

In [16]:
all_entities = [system] + [star] + moons + planets + [home_planet]
all_nodes = [b.get_data() for b in all_entities]
pd.DataFrame(all_nodes)

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,orbitsName,orbitsDistance,mass,isSupportsLife,isPopulated
0,9478383991893,Hofula,system,ordered,True,6.205,1.689,7.553,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4945593305484,Leportfield,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,NaN,NaN,NaN,NaN,NaN
2,9180000257266,Guraylarhun,moon,ice,NaN,NaN,NaN,NaN,0.182737,0571213987333,Hills,4.0060,0.007410,False,False
3,2071024525531,Koy,moon,ice,NaN,NaN,NaN,NaN,0.167772,0571213987333,Hills,4.0780,0.001767,False,False
4,9247710915138,Wi,moon,ice,NaN,NaN,NaN,NaN,0.109765,9708494056465,Rahrei,4.0001,0.020946,False,False
5,7703474315327,Jo,moon,rocky,NaN,NaN,NaN,NaN,0.185130,0571213987333,Hills,4.0001,0.000246,False,False
6,4388324383549,Bangteinwan,moon,rocky,NaN,NaN,NaN,NaN,1.469084,0571213987333,Hills,4.0630,0.000303,False,False
7,8753484490366,Maiennal,moon,rocky,NaN,NaN,NaN,NaN,1.433315,0571213987333,Hills,4.0001,0.000156,False,False
8,0697717419256,Sha,moon,ice,NaN,NaN,NaN,NaN,0.154295,9708494056465,Rahrei,4.0001,0.018628,False,False
9,7505618271831,Ky,moon,rocky,NaN,NaN,NaN,NaN,0.166931,0571213987333,Hills,4.0001,0.000262,False,False


Getting the edge values to update the graph: 

In [17]:
orbiting_bodies = [home_planet] + planets + moons

orbiting_edges = [i.get_orbits_edge() for i in orbiting_bodies]

pd.DataFrame(orbiting_edges)

,node1,node2,label,orbit_distance
0,4208695288749,4945593305484,orbits,1.0710
1,5750191789943,4945593305484,orbits,7.1870
2,9708494056465,4945593305484,orbits,6.9570
3,0571213987333,4945593305484,orbits,7.5870
4,9180000257266,0571213987333,orbits,0.0060
5,2071024525531,0571213987333,orbits,0.0780
6,9247710915138,9708494056465,orbits,0.0001
7,7703474315327,0571213987333,orbits,0.0001
8,4388324383549,0571213987333,orbits,0.0630
9,8753484490366,0571213987333,orbits,0.0001


In [18]:
pd.DataFrame(orbiting_edges).groupby(["node1", "node2"]).count()["label"].mean() == 1

True

In [19]:
pd.DataFrame(orbiting_edges).groupby(["node2"]).count()

,node1,label,orbit_distance
node2,,,
0571213987333,6,6,6
4945593305484,4,4,4
5750191789943,2,2,2
9708494056465,2,2,2


Check the orbiting logic

In [20]:
[
    f"{i.type} {i.label} orbits a {i.orbiting.type} {i.orbiting.label}"
    for i in orbiting_bodies
]

['terrestrial planet orbits a G star',
 'gas planet orbits a G star',
 'gas planet orbits a G star',
 'gas planet orbits a G star',
 'ice moon orbits a gas planet',
 'ice moon orbits a gas planet',
 'ice moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'ice moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'rocky moon orbits a gas planet',
 'rocky moon orbits a gas planet']

In [21]:
system_bodies = orbiting_bodies + [star]

system_edges = [i.get_in_system_edge() for i in system_bodies]

pd.DataFrame(system_edges)

,node1,node2,label
0,4208695288749,9478383991893,isIn
1,5750191789943,9478383991893,isIn
2,9708494056465,9478383991893,isIn
3,0571213987333,9478383991893,isIn
4,9180000257266,9478383991893,isIn
5,2071024525531,9478383991893,isIn
6,9247710915138,9478383991893,isIn
7,7703474315327,9478383991893,isIn
8,4388324383549,9478383991893,isIn
9,8753484490366,9478383991893,isIn


### Scanning the homeworld
the homeworld already has some resources known. `scan_body()` is inherited by the base object. So everyone should be able to do it. 

In [22]:
home_planet.scan_body()
home_planet.resources

[<resource: 5363345138039; Organic>,
 <resource: 1514663137619; Common Minerals>,
 <resource: 0426125345066; Rare Minerals>,
 <resource: 8995656210798; Water>]

In [23]:
pd.DataFrame([i.get_data() for i in home_planet.resources])

,name,objid,label,volume,max_volume,description,replenish_rate
0,Organic,5363345138039,resource,1091,1091,bilogical material that can be consumed by pops,10.0
1,Common Minerals,1514663137619,resource,117,117,Iron and other common material used in constru...,NaN
2,Rare Minerals,0426125345066,resource,54,54,"lithium, silver and other rare minerals used i...",NaN
3,Water,8995656210798,resource,8707,8707,"H2O ready to be consumed, either frozen or in ...",NaN


In [24]:
pd.DataFrame([i.get_location_edge() for i in home_planet.resources])

,node1,node2,label
0,4208695288749,5363345138039,has
1,4208695288749,1514663137619,has
2,4208695288749,0426125345066,has
3,4208695288749,8995656210798,has


In [25]:
[
    f"{i.location.type} {i.location.label} has {i.volume} {i.name}"
    for i in home_planet.resources
]

['terrestrial planet has 1091 Organic',
 'terrestrial planet has 117 Common Minerals',
 'terrestrial planet has 54 Rare Minerals',
 'terrestrial planet has 8707 Water']

# The Full Automated Process

Usefull as QA, to ensure that the process will run end-to-end

In [26]:
homesystem_data = universe.build_homeSystem(data, username="notebook")
pd.DataFrame(homesystem_data["nodes"])

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,...,accountid,conformity,constitution,literacy,aggression,num_planets,num_moons,starting_pop,organics,minerals
0,9779844001088,Burglas,system,ordered,True,-24.406,9.281,-0.568,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8557462460163,Ley,star,G,NaN,NaN,NaN,NaN,106.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8017543421048,Sinselraypal,moon,rocky,NaN,NaN,NaN,NaN,0.579067,9522773757925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0906172914456,Uncalsur,moon,rocky,NaN,NaN,NaN,NaN,0.028970,8789988177530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5748988067346,Laubarroc,moon,rocky,NaN,NaN,NaN,NaN,0.009487,8789988177530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9261898821488,Batmertouhills,moon,ice,NaN,NaN,NaN,NaN,0.006779,8789988177530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4751652019696,Cen,moon,rocky,NaN,NaN,NaN,NaN,0.805870,9522773757925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1359389789331,Pi,moon,terrestrial,NaN,NaN,NaN,NaN,0.993867,9522773757925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,7650676457256,Roc,moon,ice,NaN,NaN,NaN,NaN,0.148770,9522773757925,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,7096551176255,Diasaintzi,moon,rocky,NaN,NaN,NaN,NaN,0.008914,8789988177530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Check** Making sure that there are no duplicate nodes. This dataframe should be empty. 

In [27]:
dup_edges = pd.DataFrame(homesystem_data["nodes"])
dup_edges[dup_edges.duplicated(["objid"])]

,objid,name,label,class,isHomeSystem,glat,glon,gelat,radius,orbitsId,...,accountid,conformity,constitution,literacy,aggression,num_planets,num_moons,starting_pop,organics,minerals


In [28]:
type("True") == bool

False

In [29]:
print(
    f"{len(homesystem_data['edges'])}  total edges for {len(homesystem_data['nodes'])} objects"
)
pd.DataFrame(homesystem_data["edges"]).groupby("label").count()

35  total edges for 21 objects


,node1,node2,orbit_distance
label,,,
createdFrom,1,1,0
has,4,4,0
isIn,15,15,0
orbits,14,14,14
submitted,1,1,0


**Check** Making sure that there are no duplicate edges. This dataframe should be empty. 

In [30]:
dup_edges = pd.DataFrame(homesystem_data["edges"])
dup_edges[dup_edges.duplicated(["node1", "node2"])]

,node1,node2,label,orbit_distance


### Rulse For Ontology:
here is a point to check that the ontology matches this format:
* everything camelCase (starting on lower)
* Don't reference the subject in the edge label (has not hasResource)
* Every node is a Class that can be found in `objects`
* Form and Account are the exception

# The planet Surface

Entirely separate processes, the homeworld makes the population on a planet that already exists. 

In [31]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [32]:
spec = objects.species.Species(data)
spec

<species: None; 3427679415245; Wanrasgorskbia>

In [33]:
pops = [objects.population.Pop(spec) for i in range(int(data["starting_pop"]))]
pops_df = pd.DataFrame([p.get_data() for p in pops])
pops_df

,name,objid,label,conformity,literacy,aggression,constitution,health,isInFaction,industry,wealth,factionLoyalty,isIdle
0,unnamed,7347796319799,pop,0.628,0.430,0.390,0.434,0.5,None,0.4120,0.42100,0.665,True
1,unnamed,5115269404450,pop,0.482,0.545,0.542,0.381,0.5,None,0.4615,0.50325,0.407,True
2,unnamed,3332259389199,pop,0.346,0.598,0.532,0.436,0.5,None,0.4840,0.54100,0.590,True
3,unnamed,4026468861291,pop,0.456,0.446,0.454,0.523,0.5,None,0.4885,0.46725,0.493,True
4,unnamed,8878602546816,pop,0.464,0.475,0.469,0.390,0.5,None,0.4295,0.45225,0.565,True
5,unnamed,8535323141393,pop,0.414,0.698,0.554,0.651,0.5,None,0.6025,0.65025,0.316,True
6,unnamed,3558885026641,pop,0.521,0.485,0.360,0.475,0.5,None,0.4175,0.45125,0.400,True


Populations (_pops_) are born with stats that are within normal distribution for it's species. They are not in a faction and don't have names. The faction will help to determine their names. 

In [34]:
n_steps = 6
n_factions = homeworld.get_n_factions(n_steps, float(data["conformity"]))
n_factions

2

Using conformity to get the number of factions. Higher conformity, fewer factions. 

In [35]:
starting_attributes = ["conformity", "literacy", "aggression", "constitution"]

kmeans = KMeans(n_clusters=n_factions).fit(
    pops_df[[c for c in pops_df.columns if c in starting_attributes]]
)
kmeans

KMeans(n_clusters=2)

In [36]:
factions = [objects.population.Faction(i) for i in range(kmeans.n_clusters)]
factions

[<faction: None; 1803555876312; Menling>,
 <faction: None; 8567312337361; Yozufonkar>]

In [37]:
kmeans.labels_

array([0, 0, 1, 0, 0, 1, 0])

In [38]:
for i, n in enumerate(kmeans.labels_):
    pops[i].set_faction(factions[n])
    print(f"pop: {pops[i]} belongs to faction: {factions[n]}")

pop: <pop: pop; 7347796319799; Menling Springslis> belongs to faction: <faction: None; 1803555876312; Menling>
pop: <pop: pop; 5115269404450; Menling Rei> belongs to faction: <faction: None; 1803555876312; Menling>
pop: <pop: pop; 3332259389199; Yozufonkar Dinnisbynia> belongs to faction: <faction: None; 8567312337361; Yozufonkar>
pop: <pop: pop; 4026468861291; Menling Namfon> belongs to faction: <faction: None; 1803555876312; Menling>
pop: <pop: pop; 8878602546816; Menling Cotal> belongs to faction: <faction: None; 1803555876312; Menling>
pop: <pop: pop; 8535323141393; Yozufonkar Wes> belongs to faction: <faction: None; 8567312337361; Yozufonkar>
pop: <pop: pop; 3558885026641; Menling Silcon> belongs to faction: <faction: None; 1803555876312; Menling>


The name of the population should match the name of the faction. 

In [39]:
pops_df = pd.DataFrame([p.get_data() for p in pops])
pops_df

,name,objid,label,conformity,literacy,aggression,constitution,health,isInFaction,industry,wealth,factionLoyalty,isIdle
0,Menling Springslis,7347796319799,pop,0.628,0.430,0.390,0.434,0.5,1803555876312,0.4120,0.42100,0.665,True
1,Menling Rei,5115269404450,pop,0.482,0.545,0.542,0.381,0.5,1803555876312,0.4615,0.50325,0.407,True
2,Yozufonkar Dinnisbynia,3332259389199,pop,0.346,0.598,0.532,0.436,0.5,8567312337361,0.4840,0.54100,0.590,True
3,Menling Namfon,4026468861291,pop,0.456,0.446,0.454,0.523,0.5,1803555876312,0.4885,0.46725,0.493,True
4,Menling Cotal,8878602546816,pop,0.464,0.475,0.469,0.390,0.5,1803555876312,0.4295,0.45225,0.565,True
5,Yozufonkar Wes,8535323141393,pop,0.414,0.698,0.554,0.651,0.5,8567312337361,0.6025,0.65025,0.316,True
6,Menling Silcon,3558885026641,pop,0.521,0.485,0.360,0.475,0.5,1803555876312,0.4175,0.45125,0.400,True


Just as the pops reference the faction, the faction has a list of it's populations. 

In [40]:
factions[0].pops

[<pop: pop; 7347796319799; Menling Springslis>,
 <pop: pop; 5115269404450; Menling Rei>,
 <pop: pop; 4026468861291; Menling Namfon>,
 <pop: pop; 8878602546816; Menling Cotal>,
 <pop: pop; 3558885026641; Menling Silcon>]

Creating a PCA to separate the factions by distance. 

In [41]:
if n_factions >= 2:
    # using PCA to set populations on map:

    # PCA Part
    pca = PCA(n_components=2)
    X_r = pca.fit(kmeans.cluster_centers_).transform(kmeans.cluster_centers_)
    for i, f in enumerate(factions):
        f.pca_explained_variance_ratio = pca.explained_variance_ratio_
        f.lat = np.round(X_r[i][0], 3)
        f.long = np.round(X_r[i][1], 3)
else:
    # Only one faction, the lat and long is 0,0
    for i, f in enumerate(factions):
        f.lat = 0
        f.long = 0

In [42]:
pca.explained_variance_ratio_

array([1.00000000e+00, 3.55713299e-32])

In [43]:
factions[0].get_pop_edges([])

[{'node1': '7347796319799', 'node2': '1803555876312', 'label': 'isInFaction'},
 {'node1': '5115269404450', 'node2': '1803555876312', 'label': 'isInFaction'},
 {'node1': '4026468861291', 'node2': '1803555876312', 'label': 'isInFaction'},
 {'node1': '8878602546816', 'node2': '1803555876312', 'label': 'isInFaction'},
 {'node1': '3558885026641', 'node2': '1803555876312', 'label': 'isInFaction'}]

In [44]:
[p.isOfSpecies for p in pops]

[{'node1': '7347796319799', 'node2': '3427679415245', 'label': 'isOfSpecies'},
 {'node1': '5115269404450', 'node2': '3427679415245', 'label': 'isOfSpecies'},
 {'node1': '3332259389199', 'node2': '3427679415245', 'label': 'isOfSpecies'},
 {'node1': '4026468861291', 'node2': '3427679415245', 'label': 'isOfSpecies'},
 {'node1': '8878602546816', 'node2': '3427679415245', 'label': 'isOfSpecies'},
 {'node1': '8535323141393', 'node2': '3427679415245', 'label': 'isOfSpecies'},
 {'node1': '3558885026641', 'node2': '3427679415245', 'label': 'isOfSpecies'}]

Faction's pops should all be placed on a grid. `[0,0]` is the location of the faction. The pops are placed around that faction center.

In [45]:
factions[0].faction_place

[[0, 0], [0, 2], [1, 1], [2, 0], [3, -1], [-2, -2]]

In [46]:
factions[0].pops

[<pop: pop; 7347796319799; Menling Springslis>,
 <pop: pop; 5115269404450; Menling Rei>,
 <pop: pop; 4026468861291; Menling Namfon>,
 <pop: pop; 8878602546816; Menling Cotal>,
 <pop: pop; 3558885026641; Menling Silcon>]

In [47]:
len(factions[0].faction_place) - 1 == len(factions[0].pops)

True

In [48]:
pd.DataFrame([f.get_data() for f in factions])

,name,objid,label,lat,long,pop_loactions
0,Menling,1803555876312,faction,-0.129,0.0,"[[0, 0], [0, 2], [1, 1], [2, 0], [3, -1], [-2,..."
1,Yozufonkar,8567312337361,faction,0.129,0.0,"[[0, 0], [2, 2], [-1, 1]]"
